In [1]:
import os
import os.path as path
import pandas as pd
import requests
from bs4 import BeautifulSoup
import regex as re
import time

In [2]:
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'dataProjs.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('professor,projeto,resumo\n')  # Cabeçalho compatível com os dados
    print(f"Arquivo base criado ou já existente em: {basePath}")
    return basePath

In [3]:
def readArchive():
    with open(baseArchive(), 'r', encoding='utf-8') as archive:
        data = archive.readlines()
    print("Arquivo lido com sucesso!")
    return data

In [4]:
# Função para obter os resumos do projeto
def get_project_describe(esp_url):
    print(f"Obtendo resumo do projeto em: {esp_url}")
    response = requests.get(esp_url)
    if response.status_code != 200:
        print(f"Erro ao acessar curso em {esp_url}: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    projeto_resumo = []

    seccion = soup.find('article', id='conteudo')

    content_section = seccion.find('div', class_="ficha-dados cor-borda")
    if not content_section:
        print("Erro: Tabela 'tabela-dados cor-borda' não encontrada.")
        return []
    
    ficha_campo = content_section.find_all('div', class_='ficha-campo')
    for resume in ficha_campo:
        resumo = resume.text.strip()

        projeto_resumo.append({
            'resumo': resumo
        })
    print(f'coletado! resumo')

    return projeto_resumo

In [5]:
# Função para obter o nome dos projetos
def get_project_name(esp_url):
    print(f"Obtendo projetos do professor em: {esp_url}")
    response = requests.get(esp_url)
    if response.status_code != 200:
        print(f"Erro ao acessar curso em {esp_url}: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    projeto_nome = []

    content_div = soup.find('div', class_='abas-conteudo')
    projects_section = content_div.find('div', id="projetos") if content_div else None
    projects_list = projects_section.find_next('table') if projects_section else None

    if projects_list:
        projects_links = projects_list.find_all('a')  # Captura todos os links
        for link in projects_links:
            projet = link['href'].split('/')[-1]
            if projet:
                projeto = link.text.strip()
                resume_url = f'https://institucional.ufpel.edu.br{link["href"]}'

                proj_r = get_project_describe(resume_url)
                for proj in proj_r:
                    projeto_nome.append({
                        'projeto': projeto,
                        **proj
                    })
                print(f'coletado, {projeto}')

    return projeto_nome


Cursos de Graduação da Computação (Ciência e Engenharia) -> 
        Professores com lotação no Centro de Desenvolvimento Tecnológico -> 
            Projetos em andamento de Pesquisa, Ensino e Extensão -> 
                Resumo



In [6]:
def get_info(model_url):
    print(f"Obtendo informações gerais dos cursos em: {model_url}")
    response = requests.get(model_url)
    if response.status_code != 200:
        print(f"Erro ao acessar {model_url}: {response.status_code}")
        return None 

    soup = BeautifulSoup(response.content, 'html.parser')
    projetos_data = []
    codigos_desejados = ['3900', '3910']

    content_div = soup.find('div', class_='abas-conteudo')
    profs_section = content_div.find('div', id="servidores") if content_div else None
    profs_list = profs_section.find_next('table') if profs_section else None
    profs_links = profs_list.find_all('a')  # Captura todos os links

    if profs_list:
        for link in profs_links:
            next_link = link['href'].split('/')[-1]
            prof = link.find('span', class_='tabela-detalhe-titulo p-name')
            if prof and next_link:
                professor = prof.text.strip()
                projetos_url = f'https://institucional.ufpel.edu.br{link["href"]}'

                projeto_nome = get_project_name(projetos_url)
                for proj_t in projeto_nome:
                    projetos_data.append({
                        'professor': professor,
                        **proj_t
                    })
                print(f'coletado, {professor}')

    else:
        print("Nenhuma lista de professores encontrada.")

    return projetos_data


In [7]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()

    df = pd.DataFrame(data)

    try:
        # Caso o arquivo já exista
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)
            
            # Combina os novos dados com os existentes, sem remover duplicatas
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df.to_csv(filename, index=False)
        else:
            # Caso seja o primeiro salvamento
            df.to_csv(filename, index=False)

        print(f"Dados salvos no arquivo '{filename}' com sucesso!")

    except Exception as e:
        print(f"Erro ao salvar dados: {e}")


In [8]:

if __name__ == '__main__':
    url_base = 'https://institucional.ufpel.edu.br/unidades/id/443'
    data = get_info(url_base)

    if data:
        saveData(data)
    else:
        print("Nenhum dado encontrado para salvar.")

Obtendo informações gerais dos cursos em: https://institucional.ufpel.edu.br/unidades/id/443
Obtendo projetos do professor em: https://institucional.ufpel.edu.br/servidores/id/146325
Obtendo resumo do projeto em: https://institucional.ufpel.edu.br/projetos/id/u250
coletado! resumo
coletado, ExpPC Explorando o Pensamento Computacional para a Qualificação do Ensino Fundamental
Obtendo resumo do projeto em: https://institucional.ufpel.edu.br/projetos/id/u250
coletado! resumo
coletado, ExpPC Explorando o Pensamento Computacional para a Qualificação do Ensino Fundamental
coletado, ADRIANA BORDINI
Obtendo projetos do professor em: https://institucional.ufpel.edu.br/servidores/id/174884
Obtendo resumo do projeto em: https://institucional.ufpel.edu.br/projetos/id/p10239
coletado! resumo
coletado, Desenvolvimento de metodologias e de ferramentas de simulação elétrica para caracterização da confiabilidade de circuitos integrados
Obtendo resumo do projeto em: https://institucional.ufpel.edu.br/pr